## ETL-Project

In [67]:
# Import Dependencies
import pandas as pd
from sqlalchemy import create_engine
from passwords import password

In [68]:
#Extract CSVs into DataFrames
#Data sources:
    #https://data.worldbank.org/indicator/SP.DYN.LE00.IN
                #delete top 4 rows from life expectancy csv file
    #https://www.fraserinstitute.org/economic-freedom/dataset?geozone=world&page=dataset&min-year=2016&max-year=2017&filter=0&date-type=range
            #delete top 4 rows and first column from economic freedom csv file


freedom_file = "Resources/economic_freedom.csv"
expectancy_file = "Resources/life_expectancy.csv"

In [69]:
#read csvs
economic_freedom = pd.read_csv(freedom_file)
life_expectancy = pd.read_csv(expectancy_file)

In [70]:
#review first csv in Pandas
economic_freedom.head()

,Year,ISO_Code,Countries,Economic Freedom Summary Index,Rank,Government consumption,Transfers and subsidies,Government investment,Top marginal income tax rate,Top marginal income and payroll tax rate,...,Conscription,Labor market regulations,Administrative requirements,Regulatory Burden,Starting a business,Extra payments/bribes/favoritism,Licensing restrictions,Tax compliance,Business regulations,Regulation
0,2017,AGO,Angola,4.83,158,6.34,8.89,6,10,9.0,...,0,5.37,2.40,1.33,8.70,1.95,8.10,6.78,4.88,5.66
1,2017,ALB,Albania,7.67,30,8.08,7.34,8,9,7.0,...,10,6.94,6.27,6.67,9.72,4.05,6.03,7.18,6.65,7.77
2,2017,ARE,United Arab Emirates,7.17,61,4.09,8.70,2,10,10.0,...,0,6.70,6.58,5.33,9.63,8.47,10.00,9.87,8.31,7.24
3,2017,ARG,Argentina,5.67,146,5.39,6.31,6,7,1.0,...,10,5.13,2.49,7.11,9.59,3.26,5.35,6.51,5.72,5.65
4,2017,ARM,Armenia,7.70,27,7.19,7.31,8,5,5.0,...,0,6.41,4.63,6.22,9.88,4.58,9.32,7.06,6.95,7.54


In [71]:
#Filter DataFrame to keep only columns we want
economic_freedom = economic_freedom[['ISO_Code', 'Countries', 'Economic Freedom Summary Index', 'Rank']]

#rename columns to match SQL tables created in Postgres
economic_freedom = economic_freedom.rename(columns = {"ISO_Code":"iso_code","Economic Freedom Summary Index":"economic_freedom_summary_index", "Countries": "country_name", "Rank": "economic_freedom_rank"})

In [72]:
#display updated dataframe
economic_freedom

,iso_code,country_name,economic_freedom_summary_index,economic_freedom_rank
0,AGO,Angola,4.83,158
1,ALB,Albania,7.67,30
2,ARE,United Arab Emirates,7.17,61
3,ARG,Argentina,5.67,146
4,ARM,Armenia,7.70,27
...,...,...,...,...
157,VNM,Vietnam,6.27,119
158,YEM,"Yemen, Rep.",5.84,140
159,ZAF,South Africa,6.61,101
160,ZMB,Zambia,6.84,83


In [73]:
#review second csv
life_expectancy.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Aruba,ABW,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,65.662,66.074,66.444,66.787,67.113,67.435,...,75.017,75.158,75.299,75.441,75.583,75.725,75.868,76.010,NaN,NaN
1,Afghanistan,AFG,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,32.446,32.962,33.471,33.971,34.463,34.948,...,61.028,61.553,62.054,62.525,62.966,63.377,63.763,64.130,NaN,NaN
2,Angola,AGO,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,37.524,37.811,38.113,38.430,38.760,39.102,...,55.350,56.330,57.236,58.054,58.776,59.398,59.925,60.379,NaN,NaN
3,Albania,ALB,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,62.283,63.301,64.190,64.914,65.463,65.850,...,76.562,76.914,77.252,77.554,77.813,78.025,78.194,78.333,NaN,NaN
4,Andorra,AND,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
#similar to above, keep only columns we want and rename columns to match SQL
life_expectancy = life_expectancy[['Country Code', 'Country Name', '2017']]

life_expectancy = life_expectancy.rename(columns = {"2017":"life_expectancy",
                                                        "Country Name": "country_name",
                                                       "Country Code": "iso_code"})

In [75]:
#drop lines with no data
life_expectancy.dropna()

,iso_code,country_name,life_expectancy
0,ABW,Aruba,76.010000
1,AFG,Afghanistan,64.130000
2,AGO,Angola,60.379000
3,ALB,Albania,78.333000
5,ARB,Arab World,71.622526
...,...,...,...
259,XKX,Kosovo,71.946341
260,YEM,"Yemen, Rep.",66.086000
261,ZAF,South Africa,63.538000
262,ZMB,Zambia,63.043000


In [76]:
#Create database connection

USERNAME = "postgres"
DATABASE = "ETL-Project"

rds_connection_string = f"{USERNAME}:{password}@localhost:5432/{DATABASE}"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [77]:
# Confirm tables
engine.table_names()

['economic_freedom', 'life_expectancy']

In [135]:
#load life exp dataframe to SQL
life_expectancy.to_sql(name='life_expectancy', con=engine, if_exists='append', index=False)

In [134]:
# also economic freedom
economic_freedom.to_sql(name='economic_freedom', con=engine, if_exists='append', index=False)

In [78]:
#copy SQL code to merge tables
query = """select economic_freedom.iso_code,
economic_freedom.country_name,
economic_freedom.economic_freedom_summary_index,
economic_freedom.economic_freedom_rank,
life_expectancy.life_expectancy
from economic_freedom
inner join life_expectancy on
economic_freedom.iso_code = life_expectancy.iso_code"""
query_final = pd.read_sql_query(query, con=engine)
query_final

,iso_code,country_name,economic_freedom_summary_index,economic_freedom_rank,life_expectancy
0,AGO,Angola,4.83,158,60.38
1,ALB,Albania,7.67,30,78.33
2,ARE,United Arab Emirates,7.17,61,77.65
3,ARG,Argentina,5.67,146,76.37
4,ARM,Armenia,7.70,27,74.80
...,...,...,...,...,...
156,VNM,Vietnam,6.27,119,75.24
157,YEM,"Yemen, Rep.",5.84,140,66.09
158,ZAF,South Africa,6.61,101,63.54
159,ZMB,Zambia,6.84,83,63.04


In [79]:
#find avg of life expectancy for all countries
query_final.mean()

economic_freedom_summary_index     6.792050
economic_freedom_rank             81.664596
life_expectancy                   72.416273
dtype: float64

In [80]:
#sort by higher ranked countries for economic freedom
query_final.sort_values(by=['economic_freedom_rank'], ascending=True,)

,iso_code,country_name,economic_freedom_summary_index,economic_freedom_rank,life_expectancy
61,HKG,Hong Kong,8.91,1,84.68
131,SGP,Singapore,8.71,2,82.90
114,NZL,New Zealand,8.50,3,81.66
27,CHE,Switzerland,8.40,4,83.60
154,USA,United States,8.19,5,78.54
...,...,...,...,...,...
0,AGO,Angola,4.83,158,60.38
42,DZA,Algeria,4.77,159,76.50
129,SDN,Sudan,4.67,160,64.88
86,LBY,Libya,4.45,161,72.52


In [81]:
# now we'll do some binning
query_binned = query_final

bins = [0, 16, 32, 48, 64, 80, 96, 112, 128, 144, 161]
bin_names = ["0-16","17-32", "33-48", "49-64", "65-80", "81-96", "97-112", "113-128", "129-144", "145-161"]

query_binned['binned'] = pd.cut(query_binned['economic_freedom_rank'], bins, labels = bin_names)

binned_final = query_binned.groupby('binned') 

binned_final.mean()

,economic_freedom_summary_index,economic_freedom_rank,life_expectancy
binned,,,
0-16,8.163125,8.250000,80.177500
17-32,7.733333,24.733333,79.902667
33-48,7.493125,40.312500,77.141875
49-64,7.249444,56.944444,73.780556
65-80,7.001429,73.357143,74.659286
81-96,6.756250,88.125000,71.565625
97-112,6.561250,104.375000,68.927500
113-128,6.246875,120.375000,70.264375
129-144,5.918125,136.312500,64.623125
